In [1]:
import heapq

# --- 1. Helper Functions ---

# Pre-calculate the (row, col) of the goal position for each tile
# This makes the heuristic calculation very fast.
GOAL_POSITIONS = {
    1: (0, 0), 2: (0, 1), 3: (0, 2),
    4: (1, 0), 5: (1, 1), 6: (1, 2),
    7: (2, 0), 8: (2, 1), 0: (2, 2)  # 0 is the blank
}

def print_puzzle(state):
    """
    Prints the 3x3 puzzle state in a readable format.
    '0' is printed as '.'
    """
    for i in range(0, 9, 3):
        row = state[i:i+3]
        print(" ".join(str(x) if x != 0 else '.' for x in row))
    print() # Add a newline

def calculate_manhattan(state):
    """
    Calculates the Manhattan Distance heuristic (h(n)) for a state.
    """
    distance = 0
    for i in range(9):
        tile = state[i]
        if tile != 0: # We don't count the blank tile's distance
            # (current_row, current_col)
            current_row, current_col = divmod(i, 3) 
            # (goal_row, goal_col)
            goal_row, goal_col = GOAL_POSITIONS[tile]
            
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance

def get_neighbors(state):
    """
    Generates all valid neighbor states from the current state.
    """
    neighbors = []
    
    # Find the index of the blank tile (0)
    blank_index = state.index(0)
    
    # Get (row, col) of the blank tile
    row, col = divmod(blank_index, 3)
    
    # (dr, dc) = (delta_row, delta_col)
    # Check moves: Up, Down, Left, Right
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nr, nc = row + dr, col + dc
        
        # Check if the move is within the 3x3 grid
        if 0 <= nr < 3 and 0 <= nc < 3:
            
            # Find the linear index of the tile to swap with
            new_index = nr * 3 + nc
            
            # Create the new state by swapping
            new_state_list = list(state)
            new_state_list[blank_index], new_state_list[new_index] = \
                new_state_list[new_index], new_state_list[blank_index]
            
            neighbors.append(tuple(new_state_list))
            
    return neighbors

# --- 2. A* Solver ---

def solve_8_puzzle(start_state, goal_state):
    """
    Solves the 8-puzzle game using A* search.
    Returns the list of states in the shortest path.
    """
    
    # --- A* Setup ---
    # The priority queue stores: (f_cost, g_cost, state, path_list)
    # We use a tuple (state) as the key because it's hashable.
    
    g_cost = 0 # Moves so far
    h_cost = calculate_manhattan(start_state) # Estimated moves left
    f_cost = g_cost + h_cost
    
    # The priority queue
    open_set = [(f_cost, g_cost, start_state, [start_state])]
    
    # g_costs dictionary stores the *cheapest* g_cost found so far
    # to reach a state. This also acts as our 'visited' set.
    g_costs = {start_state: 0}

    # --- A* Loop ---
    while open_set:
        # Get the node with the *lowest f_cost*
        f, g, current_state, path = heapq.heappop(open_set)
        
        # --- GOAL CHECK ---
        if current_state == goal_state:
            return path  # We found the shortest solution!

        # --- EXPLORE NEIGHBORS ---
        for neighbor in get_neighbors(current_state):
            
            # new_g is the cost to reach this neighbor
            # (cost of 1 move from the current state)
            new_g = g + 1
            
            # Check if this neighbor is new, OR if we found a
            # *shorter path* to it than before.
            if neighbor not in g_costs or new_g < g_costs[neighbor]:
                
                # Update the g_cost for this state
                g_costs[neighbor] = new_g
                
                # Calculate new h_cost and f_cost
                new_h = calculate_manhattan(neighbor)
                new_f = new_g + new_h
                
                # Add the new path to the priority queue
                new_path = path + [neighbor]
                heapq.heappush(open_set, (new_f, new_g, neighbor, new_path))
                
    return None # No solution found

# --- 3. Main part of the script ---

if __name__ == "__main__":
    
    # 0 represents the blank space
    # We use tuples because they are hashable (can be used as dict keys)
    start_state = (1, 8, 2, 0, 4, 3, 7, 6, 5)
    goal_state  = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    
    print("--- 8-Puzzle Solver using A* ---")
    print("Start State:")
    print_puzzle(start_state)
    
    print("Goal State:")
    print_puzzle(goal_state)
    
    print("Solving...")
    
    # Run the solver
    path = solve_8_puzzle(start_state, goal_state)
    
    # --- Print the results ---
    if path:
        print(f"Solution found in {len(path) - 1} moves!")
        print("\n--- Solution Path ---")
        
        # Loop through the path and print each step
        for i, state in enumerate(path):
            print(f"Move {i}:")
            print_puzzle(state)
            
    else:
        print("No solution found.")

--- 8-Puzzle Solver using A* ---
Start State:
1 8 2
. 4 3
7 6 5

Goal State:
1 2 3
4 5 6
7 8 .

Solving...
Solution found in 9 moves!

--- Solution Path ---
Move 0:
1 8 2
. 4 3
7 6 5

Move 1:
1 8 2
4 . 3
7 6 5

Move 2:
1 . 2
4 8 3
7 6 5

Move 3:
1 2 .
4 8 3
7 6 5

Move 4:
1 2 3
4 8 .
7 6 5

Move 5:
1 2 3
4 8 5
7 6 .

Move 6:
1 2 3
4 8 5
7 . 6

Move 7:
1 2 3
4 . 5
7 8 6

Move 8:
1 2 3
4 5 .
7 8 6

Move 9:
1 2 3
4 5 6
7 8 .

